In [30]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
PINECONE_API_KEY="ac7fbe02-b89d-4224-914f-3ad2181a1054"
PINECONE_API_ENV="us-east-1"

In [3]:
#extract pdf
def load_pdf(data):
    directory_loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=directory_loader.load()

    return documents

In [4]:
extracted_data=load_pdf("data/")

In [5]:
#divide text into chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks=text_split(extracted_data)
print("lenght og my chunk",len(text_chunks))

lenght og my chunk 5860


In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [8]:
try:
    embeddings = download_hugging_face_embeddings()
except Exception as e:
    print(f"An error occurred: {e}")

C:\Users\NAVNEET GUGLANI\AppData\Local\Temp\ipykernel_10612\2535699610.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
query_result

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [13]:
from pinecone import Pinecone, ServerlessSpec
import time
from tqdm import tqdm  # for progress tracking

# Initialize Pinecone
pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name = "medical-chatbot"

# Check if index exists, if not create it
if index_name not in pc.list_indexes().names():
    spec = ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
    
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=spec
    )
    # Wait for index to be ready
    time.sleep(20)

# Get the index
index = pc.Index(index_name)

# Create embeddings and upsert to Pinecone in smaller batches
batch_size = 100  # Adjust based on your needs
embeddings_list = []
total_vectors = 0

print(f"Total chunks to process: {len(text_chunks)}")

# Process in batches
for i in tqdm(range(0, len(text_chunks), batch_size)):
    batch = text_chunks[i:i + batch_size]
    
    # Create embeddings for the batch
    batch_embeddings = []
    for chunk in batch:
        try:
            vector = embeddings.embed_query(chunk.page_content)
            batch_embeddings.append(vector)
        except Exception as e:
            print(f"Error creating embedding: {e}")
            continue
    
    # Create vectors for the batch
    vectors = list(zip(
        [str(j) for j in range(i, i + len(batch_embeddings))],
        batch_embeddings,
        [{"text": chunk.page_content} for chunk in batch[:len(batch_embeddings)]]
    ))
    
    # Upsert batch
    try:
        index.upsert(vectors=vectors)
        total_vectors += len(vectors)
        print(f"Upserted batch of {len(vectors)} vectors. Total vectors: {total_vectors}")
        
        # Add a small delay between batches
        time.sleep(1)
    except Exception as e:
        print(f"Error upserting batch: {e}")
    
    # Verify the count periodically
    if i % (batch_size * 5) == 0:
        try:
            stats = index.describe_index_stats()
            print(f"Current index stats: {stats}")
        except Exception as e:
            print(f"Error getting index stats: {e}")

# Final verification
try:
    final_stats = index.describe_index_stats()
    print(f"Final index stats: {final_stats}")
except Exception as e:
    print(f"Error getting final stats: {e}")

Total chunks to process: 5860


  0%|          | 0/59 [00:00<?, ?it/s]

Upserted batch of 100 vectors. Total vectors: 100


  2%|▏         | 1/59 [00:34<33:33, 34.72s/it]

Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5860}},
 'total_vector_count': 5860}
Upserted batch of 100 vectors. Total vectors: 200


...

Upserted batch of 100 vectors. Total vectors: 5800


 98%|█████████▊| 58/59 [07:42<00:07,  7.19s/it]

Upserted batch of 60 vectors. Total vectors: 5860


100%|██████████| 59/59 [07:48<00:00,  7.94s/it]


Final index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5860}},
 'total_vector_count': 5860}


In [35]:
import os
from pinecone import Pinecone as PineconeClient

PINECONE_API_KEY="ac7fbe02-b89d-4224-914f-3ad2181a1054"
PINECONE_API_ENV="us-east-1"

# Initialize Pinecone client
pc = PineconeClient(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

# Connect to existing index
index = pc.Index(index_name)
query="what are allergies"

# Create query embedding
query_embedding = embeddings.embed_query(query)

# Perform similarity search
search_results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

# Print results
print("Results:")
for i, match in enumerate(search_results['matches'], 1):
    print(f"\nResult {i}:")
    print(f"Score: {match['score']}")
    print(f"Text: {match['metadata']['text']}")

Results:

Result 1:
Score: 0.709312677
Text: Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying molecules, or antigens, on the
foreign organisms. This reaction between antibody and
antigen sets off a series of reactions designed to protect
the body from infection. Sometimes, this same series of

Result 2:
Score: 0.686668158
Text: reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic conjunctivitis) causes redness,
irritation, and increased tearing in the eyes. Asthma caus-
es wheezing, coughing, and shortness of breath. Symp-
toms of food allergies depend on the tissues most sensi-
tive to the allergen and whether the allergen spread sys-

Result 3:
Sco

In [38]:
import os
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pinecone import Pinecone as PineconeClient
from langchain_openai import OpenAI  # or whatever LLM you're using

# Set your API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

# Initialize your LLM
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

# Optional: Define a custom prompt
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

# Then continue with the code above...

In [45]:
import os
from pinecone import Pinecone as PineconeClient
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA

# Method 1: Set environment variables
PINECONE_API_KEY = "ac7fbe02-b89d-4224-914f-3ad2181a1054"
PINECONE_API_ENV = "us-east-1"

# Set them in environment
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_ENV"] = PINECONE_API_ENV

# Initialize Pinecone client
pc = PineconeClient(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name = "medical-chatbot"

# Verify Pinecone connection
try:
    print("Available indices:", pc.list_indexes().names())
except Exception as e:
    print(f"Error connecting to Pinecone: {e}")

# Create vector store
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
    text_key="text"
)

# Create QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

# Test the system
query = "What are Allergies?"
result = qa({"query": query})
print("Answer:", result['result'])

Available indices: ['medical-chatbot', 'testing']


C:\Users\NAVNEET GUGLANI\AppData\Local\Temp\ipykernel_10612\292047435.py:46: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


Answer: Allergies are a condition where the body's immune system overreacts to harmless substances, such as pollen, dust, or certain foods. This overreaction can cause a variety of symptoms, including itchy eyes, runny nose, wheezing, coughing, and shortness of breath. Allergies are caused by the body's immune system mistakenly identifying these harmless substances as threats, and attempting to fight them off.


In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a skin disorder in which the sebaceous glands become inflamed.


In [1]:
import langchain
print(langchain.__version__)

0.3.7
